In [49]:
# import and display options
%matplotlib notebook
from pathlib import Path

import cv2
import ipywidgets as widgets
import numpy as np
import pandas as pd
from dateutil.parser import parse
from IPython.display import display, HTML
from ipywidgets import interact, IntProgress, Label, VBox, HBox
from matplotlib import pyplot as plt

display(HTML("<style>.container {width:85% !important;}</style>"))

In [55]:
# hardcoded data directory and options for both serial titles and metadata fields
data_directory_path = Path('data/')
serial_titles_dict = {'Select a title': 'Select a title',
                      'Tennessee farm news': 'agrtfn',
                      'Tennessee farm and home science': 'agrtfhs',
                      'UT Special extension circulars': 'agrutesc'
                     }
metadata_fields_list = ['Select a metadata field', 'date', 'title']

In [56]:
# create interactive csv_path widget
identifier_widget = widgets.RadioButtons(layout={'width': 'initial'},
                                         style={
                                             'description_width': 'initial'},
                                         options=serial_titles_dict,
                                         description='Serial Title:',
                                         disabled=False
                                        )
metadata_field_widget = widgets.RadioButtons(layout={'width': 'initial'},
                                            style={
                                                'description_width': 'initial'},
                                            options=metadata_fields_list,
                                            description='Metadata field:',
                                            disabled=False,
                                           )

csv_path_display_widget = widgets.Text(layout={'width': 'initial'},
                                       description='CSV Path:')

csv_path_exists_validity_widget = widgets.Valid(description='Does CSV exist?',
                                               style={
                                                'description_width': 'initial'})
load_csv_button_widget = widgets.Button(description='Load CSV',
                                        style={'description_width': 'initial'},
                                       )

load_csv_button_output_widget = widgets.Text(layout={'width': 'initial'})


def on_button_clicked(b):
    global metadata
    # csv_path = Path(csv_path_display_widget.value)
    try:
        dataframe = pd.read_csv(csv_path_display_widget.value)
    except FileNotFoundError:
        load_csv_button_output_widget.value = 'No dataframe; CSV path invalid'
        return
    metadata = MetadataField(csv_path_display_widget.value)
    number_of_rows, number_of_columns = dataframe.shape
    load_csv_button_output_widget.value = f'{metadata.csv_path.name} loaded as dataframe with {metadata.number_of_rows} rows and {metadata.number_of_columns} columns'
    return metadata

load_csv_button_widget.on_click(on_button_clicked)

def select_csv(identifier, metadata_field):
    csv_name = f'{identifier}_{metadata_field}.csv'
    csv_path = data_directory_path.joinpath(csv_name)
    csv_path_display_widget.value = str(csv_path.resolve())
    csv_path_exists_validity_widget.value = csv_path.is_file()
    # print(f'Path to CSV: {csv_path}')
    return csv_path


# csv_path_interactive_widget = widgets.interactive(select_csv, identifier=identifier_widget, metadata_field=metadata_field_widget)
# output_csv_path_widget = interactive(select_csv, 'identifier'=identifier_widget, 'metadata_field'=metadata_field_widget)
csv_path_interactive_widget = widgets.interactive_output(select_csv, {'identifier': identifier_widget, 'metadata_field': metadata_field_widget})

In [57]:
class MetadataField():
    def __init__(self, csv_path):
        self.csv_path = Path(csv_path)
        self.dataframe = pd.read_csv(self.csv_path)
        self.number_of_rows, self.number_of_columns = self.dataframe.shape      

In [58]:
# Select a title and metadata field to process
row_1_widgets = HBox([identifier_widget, metadata_field_widget])

row_2_widgets = csv_path_display_widget  # HBox([csv_path_display_widget])

row_3_widgets = HBox([load_csv_button_widget, csv_path_exists_validity_widget])

row_4_widgets = load_csv_button_output_widget

my_widget = VBox([row_1_widgets, row_2_widgets, row_3_widgets, row_4_widgets])

my_widget

In [59]:
metadata.dataframe.columns

Index(['date_crop_box', 'date_guess', 'image_id', 'image_name'], dtype='object')

In [38]:
# write images_not_processed_list to text file QUICKLY with Python!
output_path = Path('/Users/jeremy/Documents/GitHub/utk_ProjectCeres/data/agrtfn_date_to_process.txt')
with open (output_path, 'w') as text_file:
    text_file.write('\n'.join(images_not_processed_list))

In [60]:
image_names_list = metadata.dataframe.image_name.unique().tolist()
len(image_names_list)

3062

In [ ]:
# create interactive date selection widget
months_dict = {'January': 1,
               'February': 2,
               'March': 3,
               'April': 4,
               'May': 5,
               'June': 6,
               'July': 7,
               'August': 8,
               'September': 9,
               'October': 10,
               'November': 11,
               'December': 12
              }

numbers_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

month_widget = widgets.RadioButtons(options=months_dict,
                                   description='Month:')

day_tens_widget = widgets.RadioButtons(options=numbers_list,
                                      description='Day 10s:')
day_ones_widget = widgets.RadioButtons(options=numbers_list,
                                      description='Day 1s:')
year_tens_widget = widgets.RadioButtons(options=number_list,
                                       description='Year 10s:')
year_ones_widget = widgets.RadioButtons(options=numbers_list,
                                      description='Year 1s:')

date_display_widget = widgets.Text(description='Date:')


def select_date(month, day_tens, day_ones, year_tens, year_ones):
    date = []

    
select_date_interactive_widget = widgets.interactive_output(select_date, {'month': month_widget,
                                                                          'day_tens': day_tens_widget,
                                                                          'day_ones': day_ones_widget,
                                                                          'year_tens': year_tens_widget,
                                                                          'year_ones': year_ones_widget}
                                                           )

# load_csv_button_widget = widgets.Button(description='Load CSV',
#                                         style={'description_width': 'initial'},
#                                        )

# load_csv_button_output_widget = widgets.Text(layout={'width': 'initial'})
    
# def on_button_clicked(b):
#     global metadata
#     # csv_path = Path(csv_path_display_widget.value)
#     try:
#         dataframe = pd.read_csv(csv_path_display_widget.value)
#     except FileNotFoundError:
#         load_csv_button_output_widget.value = 'No dataframe; CSV path invalid'
#         return
#     metadata = MetadataField(csv_path_display_widget.value)
#     number_of_rows, number_of_columns = dataframe.shape
#     load_csv_button_output_widget.value = f'{metadata.csv_path.name} loaded as dataframe with {metadata.number_of_rows} rows and {metadata.number_of_columns} columns'
#     return metadata

# load_csv_button_widget.on_click(on_button_clicked)

# def select_csv(identifier, metadata_field):
#     csv_name = f'{identifier}_{metadata_field}.csv'
#     csv_path = data_directory_path.joinpath(csv_name)
#     csv_path_display_widget.value = str(csv_path.resolve())
#     csv_path_exists_validity_widget.value = csv_path.is_file()
#     # print(f'Path to CSV: {csv_path}')
#     return csv_path


# csv_path_interactive_widget = widgets.interactive(select_csv, identifier=identifier_widget, metadata_field=metadata_field_widget)
# output_csv_path_widget = interactive(select_csv, 'identifier'=identifier_widget, 'metadata_field'=metadata_field_widget)
# csv_path_interactive_widget = widgets.interactive_output(select_csv, {'identifier': identifier_widget, 'metadata_field': metadata_field_widget})

In [48]:
# process dates/images

# images currently on external hard drive
image_data_directory_path = Path('/Volumes/jmoor167/data/agrtfn/')

# get unique list of image names to process
image_names_list = metadata.dataframe['image_name'].unique().tolist()

for image_name in image_names_list:
    
    # get a dataframe for all rows with image_name
    image_name_dataframe = metadata.dataframe[metadata.dataframe['image_name'] == image_name]
    
    # set image_path and load image
    image_path = image_data_directory_path.joinpath(image_name)
    image = cv2.imread(str(image_path))
    
    # 
    
    # get number of image_ids -- to be later renamed date_guess_id
    number_of_date_guess = 2

1
1
1
1
4
1
1
2
2
1
1
1
1
1
2
3
2
1


KeyboardInterrupt: 